# Notebook 03: Pipeline Fundamentals

Here, we dive into Semantiva’s pipeline architecture and how to structure larger workflows.

**Topics covered:**
- Introducing context operations like renaming and deleting context items
- Understanding the dual-channel concept (data vs. context)
- Building pipelines with `AlgorithmNode` and `ProbeNode`
- Executing pipelines in both static and data-driven modes
- Configuring pipelines via YAML
- Inspecting and benchmarking pipelines

By the end, you’ll be able to design and run your own Semantiva pipelines with confidence.

---

1. **Pipelines: Concept and Structure**
    
    - Separation between data and context (dual-channel flow)
    - Definition and benefits
2. **Introducing Context Operations**
    
    - Context renamer and context deleter
    - Why they are useful in image processing workflows
3. **Nodes**
    
    - `AlgorithmNode`
    - `ProbeNode`
        - `ProbeContextInjectorNode`
        - `ProbeResultCollectorNode`
4. **Pipeline Execution Flow**
    
    - From static operations to data-driven operations
    - Step-by-step pipeline run
5. **Defining a Pipeline**
    
    - Combining examples from previous notebooks
    - Pipeline configuration in YAML
6. **Pipeline Inspection and Timers**
    
    - Pipeline inspection
    - Benchmarking
    - Pipeline execution in debug mode 

> **Goal**: Show how to wire together algorithms, probes into reusable, systematic workflows.